In [1]:
import cv2
import numpy as np
import os
import glob
from matplotlib import pyplot as plt

def plot_image(orig, edge):
    plt.figure(figsize=(10, 5))
    plt.subplot(121), plt.imshow(orig, cmap='gray')
    plt.title('Original Image'), plt.xticks([]), plt.yticks([])
    plt.subplot(122), plt.imshow(edge, cmap='gray')
    plt.title('Edge Image'), plt.xticks([]), plt.yticks([])
    plt.show()
    
def preprocess_image(image_path, cp, target_size=(128, 128)):
    orig_image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    blurred_image = cv2.GaussianBlur(orig_image, (5, 5), 1.4)
    edges = cv2.Canny(blurred_image, cp['threshold1'], cp['threshold2'], apertureSize=cp['apertureSize'])
    if edges.shape[:2] != target_size:
        edges = cv2.resize(edges, target_size)
    #plot_image(orig_image, edges)
    
    return edges

def extract_hog_features(image, hog_descriptor):
    return hog_descriptor.compute(image)

def load_data(image_paths, hog_descriptor, cp, categories):    
    print("Started extracting descriptors...")
    hog_descriptors_list = []
    labels_list = []
    for image_path in image_paths:
        # print(f"Processing: {image_path}")
        category = os.path.basename(os.path.dirname(image_path))
        image = preprocess_image(image_path, cp, target_size=hog_descriptor.winSize)
        hog_descriptors = extract_hog_features(image, hog_descriptor)
        hog_descriptors_list.append(hog_descriptors)
        labels = np.zeros(len(categories))
        labels[categories.index(category)] = 1
        labels_list.append(labels)
    X = np.array(hog_descriptors_list, dtype=np.float32)
    X = X.reshape(X.shape[0], -1) # Reshape the feature array to (num_samples, num_features)
    print("Completed extracting descriptors.")
    print(f"X.shape = {X.shape}")
    return X, labels_list

In [19]:
canny_params = [
    {'threshold1': 50, 'threshold2': 150, 'apertureSize': 3},
    {'threshold1': 100, 'threshold2': 200, 'apertureSize': 3},
    {'threshold1': 150, 'threshold2': 250, 'apertureSize': 5}
]

hog_params = [
    {'_blockSize':(16, 16), '_blockStride':(8, 8),'_cellSize':(8, 8),'_nbins':9},
    {'_blockSize':(16, 16), '_blockStride':(4, 4),'_cellSize':(4, 4),'_nbins':9},
    {'_blockSize':(16, 16), '_blockStride':(8, 8),'_cellSize':(8, 8),'_nbins':12},
    {'_blockSize':(16, 16), '_blockStride':(4, 4),'_cellSize':(4, 4),'_nbins':12},
]

image_files = glob.glob(f"First50/*/*.jpg")
print(f"image files: {len(image_files)}")

categories = []
for image_path in image_files:
    category = os.path.basename(os.path.dirname(image_path))
    if category not in categories:
        categories.append(category)
print(f'categories: {len(categories)}')

feature_sets = []
for cp in canny_params:
    for hp in hog_params:
        hog_descriptor = cv2.HOGDescriptor(
            _winSize=(128, 128),
            _blockSize=hp['_blockSize'],
            _blockStride=hp['_blockStride'],
            _cellSize=hp['_cellSize'],
            _nbins=hp['_nbins']
        )
        
        X, labels_list = load_data(image_files, hog_descriptor, cp, categories)
        feature_sets.append({
                    'canny_params': cp,
                    'hog_params': hp,
                    'descriptors': X,
                    'labels_list': labels_list
                })

image files: 5083
categories: 50
Started extracting descriptors...
Completed extracting descriptors.
X.shape = (5083, 8100)
Started extracting descriptors...
Completed extracting descriptors.
X.shape = (5083, 121104)
Started extracting descriptors...
Completed extracting descriptors.
X.shape = (5083, 10800)
Started extracting descriptors...
Completed extracting descriptors.
X.shape = (5083, 161472)
Started extracting descriptors...
Completed extracting descriptors.
X.shape = (5083, 8100)
Started extracting descriptors...
Completed extracting descriptors.
X.shape = (5083, 121104)
Started extracting descriptors...
Completed extracting descriptors.
X.shape = (5083, 10800)
Started extracting descriptors...
Completed extracting descriptors.
X.shape = (5083, 161472)
Started extracting descriptors...
Completed extracting descriptors.
X.shape = (5083, 8100)
Started extracting descriptors...
Completed extracting descriptors.
X.shape = (5083, 121104)
Started extracting descriptors...
Completed e

In [20]:
#
# Proof of train/test stratified sampling over each category
#
# Train : Test = 80 : 20 ==> Test sample count / Train should be equal to approx 0.25 for each category
#
print(sum(y_test)/sum(y_train))

[0.25       0.25       0.23529412 0.23529412 0.25201072 0.23684211
 0.25581395 0.24324324 0.26923077 0.25490196 0.25641026 0.26470588
 0.25       0.24657534 0.25       0.26470588 0.25510204 0.23684211
 0.25531915 0.24       0.24418605 0.23684211 0.25454545 0.25862069
 0.25       0.25       0.24390244 0.23913043 0.24074074 0.23809524
 0.25       0.25925926 0.25       0.25490196 0.26190476 0.25490196
 0.25       0.25       0.25       0.24074074 0.24074074 0.25
 0.25925926 0.25925926 0.24390244 0.25316456 0.25       0.23529412
 0.25581395 0.25714286]


In [24]:
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.svm import SVC
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import accuracy_score, classification_report

best_acc = 0
best_feature_set = None
best_model = None
best_model_hyperparams = None
for feature_set in feature_sets:
    print(f"canny_params: {feature_set['canny_params']}")
    print(f"hog_params: {feature_set['hog_params']}")
    
    stratify_labels = np.argmax(feature_set['labels_list'], axis=1)
    X_train, X_test, y_train, y_test = train_test_split(feature_set['descriptors'], stratify_labels,#feature_sets[0]['labels_list'], 
                                                    test_size=0.2, random_state=42, stratify=stratify_labels)

    # Define parameter grid for Grid Search
    param_grid = {
        'C': [0.1],#, 1, 10],
        'kernel': ['linear', 'rbf'],
        'gamma': ['scale']#, 'auto']
    }
    
    svm = SVC(decision_function_shape='ovr')  # 'ovr' stands for one-vs-rest
    kf = KFold(n_splits=3, shuffle=True, random_state=42)
    
    grid_search = GridSearchCV(svm, param_grid, cv=kf, scoring='accuracy', n_jobs=-1, verbose=10)
    grid_search.fit(X_train, y_train)
    best_int_model = grid_search.best_estimator_
    print("Best hyperparameters found: ", grid_search.best_params_)

    # Evaluate on the test set
    y_test_pred = best_int_model.predict(X_test)
    test_accuracy = accuracy_score(y_test, y_test_pred)
    print(f'Test Accuracy: {test_accuracy:.2f}')
    if test_accuracy > best_acc:
        print("*** Updated best model / feature set***")
        best_acc = test_accuracy
        best_feature_set = feature_set
        best_model = best_int_model
        best_model_hyperparams = grid_search.best_params_

print('Test Classification Report:')
print(classification_report(y_test, y_test_pred))

canny_params: {'threshold1': 50, 'threshold2': 150, 'apertureSize': 3}
hog_params: {'_blockSize': (16, 16), '_blockStride': (8, 8), '_cellSize': (8, 8), '_nbins': 9}
Fitting 3 folds for each of 2 candidates, totalling 6 fits
Best hyperparameters found:  {'C': 0.1, 'gamma': 'scale', 'kernel': 'linear'}
Test Accuracy: 0.62
*** Updated best model / feature set***
canny_params: {'threshold1': 50, 'threshold2': 150, 'apertureSize': 3}
hog_params: {'_blockSize': (16, 16), '_blockStride': (4, 4), '_cellSize': (4, 4), '_nbins': 9}
Fitting 3 folds for each of 2 candidates, totalling 6 fits
Best hyperparameters found:  {'C': 0.1, 'gamma': 'scale', 'kernel': 'linear'}
Test Accuracy: 0.63
*** Updated best model / feature set***
canny_params: {'threshold1': 50, 'threshold2': 150, 'apertureSize': 3}
hog_params: {'_blockSize': (16, 16), '_blockStride': (8, 8), '_cellSize': (8, 8), '_nbins': 12}
Fitting 3 folds for each of 2 candidates, totalling 6 fits
Best hyperparameters found:  {'C': 0.1, 'gamma':

C:\Users\sseksaria\.conda\envs\py311-cv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\sseksaria\.conda\envs\py311-cv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\sseksaria\.conda\envs\py311-cv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(resu